In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\swath\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\swath\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\swath\anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\swath\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\swath\anaconda3\Lib\site-packages\

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [13]:
re=grid.cv_results_
print(re)
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

{'mean_fit_time': array([1.62901878e-03, 3.37171555e-03, 4.73730087e-02, 3.33874893e-01,
       3.54927063e-02, 3.07159328e-01, 1.68180466e-04, 1.12309456e-03,
       3.02642345e-02, 2.95522833e-01, 2.92851448e-02, 2.36245298e-01]), 'std_fit_time': array([0.00141073, 0.0008837 , 0.00212158, 0.01829373, 0.00350407,
       0.01195517, 0.00021785, 0.00125806, 0.00334621, 0.00991058,
       0.00309227, 0.01712077]), 'mean_score_time': array([0.        , 0.        , 0.00776014, 0.01384177, 0.00616598,
       0.01728935, 0.        , 0.        , 0.006322  , 0.0147016 ,
       0.00783553, 0.00822473]), 'std_score_time': array([0.        , 0.        , 0.00209367, 0.00283313, 0.00187021,
       0.00394516, 0.        , 0.        , 0.00398424, 0.00368527,
       0.005059  , 0.00255235]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                   'entropy'],
             mask=[Fal

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100}: 0.9108102345415777


In [18]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[78  7]
 [ 5 44]]


In [20]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93        85
           1       0.86      0.90      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.90       134
weighted avg       0.91      0.91      0.91       134



In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9561824729891957

In [24]:
table=pd.DataFrame.from_dict(re)

In [26]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001629,0.001411,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.003372,0.000884,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.047373,0.002122,0.007760,0.002094,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.764706,0.810811,0.780488,0.900000,0.914286,0.834058,0.061652,5
3,0.333875,0.018294,0.013842,0.002833,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.800000,0.842105,0.809524,0.900000,0.923077,0.854941,0.048824,3
4,0.035493,0.003504,0.006166,0.001870,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.687500,0.777778,0.750000,0.871795,0.914286,0.800272,0.082299,8
5,0.307159,0.011955,0.017289,0.003945,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.800000,0.871795,0.809524,0.900000,0.947368,0.865737,0.055418,1
6,0.000168,0.000218,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001123,0.001258,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.030264,0.003346,0.006322,0.003984,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.764706,0.820513,0.761905,0.871795,0.888889,0.821561,0.052629,7
9,0.295523,0.009911,0.014702,0.003685,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.800000,0.842105,0.809524,0.900000,0.947368,0.859799,0.056038,2


In [33]:
age = float(input("Enter Age: "))
estimated_salary = float(input("Enter Estimated Salary: "))
gender_male = int(input("Enter Gender (Male=1, Female=0): "))

Enter Age:  19
Enter Estimated Salary:  19000
Enter Gender (Male=1, Female=0):  1


In [35]:
Future_Prediction=grid.predict([[age,estimated_salary,gender_male]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
